# Instalacja GLPK

W pierwszej kolejności należy wykonać **instalację** GNU Linear Programming Kit, np. wg oficjalnych [instrukcji](https://www.gnu.org/software/glpk/).

Poniżej przedstawiono proces instalacji w systemie Linux na potrzeby niniejszego notatnika.

Proces kompilacji może potrwać ok 2 do 3 minut.

In [5]:
!wget -N https://ftp.gnu.org/gnu/glpk/glpk-5.0.tar.gz
!tar -xf glpk-5.0.tar.gz
# Kompilacja i instalacja
!echo "Rozpoczynam kompilację. Może to potrwać kilka minut..."
!cd glpk-5.0/ && ./configure 1> /dev/null && make 1> /dev/null && make install 1> /dev/null
# Rejestracja biblioteki libglpk
!echo /usr/local/lib/libglpk.so.40 | tee /etc/ld.so.conf.d/libglpk.con && ldconfig
!echo "Zakończone"


# Definiujemy nowe "cell magic" dla uproszczenia uruchamiania solvera Glpsol

import re
import tempfile
import subprocess
from pathlib import Path
from IPython import get_ipython
from IPython.display import HTML, display
from IPython.core.magic import Magics, cell_magic, magics_class
import shlex
import html
import logging

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

HIGHLIGHT_RULES = {
    'color:green': re.compile(r'\b((?:INTEGER)?\s*OPTIMAL[\s\w]*?\n)\b', re.IGNORECASE),
    'color:red': re.compile(r'\b(INFEASIBLE|UNBOUNDED)\b', re.IGNORECASE),
    'color:#FF0000': re.compile(r'\b(MathProg model processing error)\b', re.IGNORECASE),
    'color:#c76e00': re.compile(r'((?:\bobjective:.*?\n)|(?:\bDisplay\sstatement.*?\n))', re.IGNORECASE),
}

def syntax_highlight(text):
    """
    Applies syntax highlighting to the glpsol output by wrapping matched patterns with span tags.
    """
    # Escape HTML special characters
    escaped_text = html.escape(text)

    # Apply highlighting rules
    for style, pattern in HIGHLIGHT_RULES.items():
        replacement = rf'<span style="{style}">\1</span>'
        escaped_text = pattern.sub(replacement, escaped_text)

    # Return the formatted HTML
    return f'<pre style="max-width: 800px; white-space: pre-wrap; font-family: monospace;">{escaped_text}</pre>'


@magics_class
class GlpkMagic(Magics):
    @cell_magic
    def glpsol(self, line, cell):
        """
        Cell magic to run glpsol on the provided LP model.

        Usage:
        %%glpsol [glpsol options]
        <LP model>
        """
        lp_file = None
        sol_file = None
        try:
            with tempfile.TemporaryDirectory() as tmpdir:
                tmp_path = Path(tmpdir)
                lp_file = tmp_path / "model.lp"
                sol_file = tmp_path / "model.sol"

                # Write the LP model to the temporary .lp file
                lp_file.write_text(cell, encoding='utf-8')

                # Parse command-line options safely
                cmd_options = shlex.split(line)
                cmd = ['glpsol'] + cmd_options + ['-m', str(lp_file), '-o', str(sol_file)]

                # Run glpsol and capture output
                result = subprocess.run(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True,
                    check=True
                )

                output = result.stdout

                # Check if the solution file exists and read its content
                if sol_file.exists():
                    sol_content = sol_file.read_text(encoding='utf-8')
                    output += ('\n' + '-' * 80 + '\n') + sol_content
                else:
                    logger.warning(f"Expected solution file {sol_file} not found.")

        except subprocess.CalledProcessError as e:
            # Capture and process error output
            output = e.stdout + '\n' + e.stderr
            processed_output = self._process_error_output(output, cell, lp_file)
            highlighted = syntax_highlight(processed_output)
            display(HTML(highlighted))
            raise RuntimeError(processed_output) from None
        except Exception as e:
            # Handle unexpected exceptions
            logger.error("An unexpected error occurred while running glpsol.", exc_info=True)
            display(HTML(f'<span style="color:red;">An unexpected error occurred: {e}</span>'))
            return

        # Apply syntax highlighting and display
        highlighted = syntax_highlight(output)
        display(HTML(highlighted))

    def _process_error_output(self, output, cell, lp_file):
        """
        Processes glpsol error output to map errors back to the LP model lines.
        """
        processed = []
        input_lines = cell.split('\n')
        lp_filename = lp_file.name if lp_file else 'model.lp'

        for ln in output.split('\n'):
            if ln.startswith(f"{lp_filename}:"):
                processed.append(f'\n{ln}')
                tail = ln[len(lp_filename) + 1:]
                match = re.match(r"^\d+", tail)
                if match:
                    orig_line_index = int(match.group()) - 1
                    if 0 <= orig_line_index < len(input_lines):
                        processed.append(input_lines[orig_line_index])
            else:
                processed.append(ln)

        return '\n'.join(processed)

# Register the magic with IPython
ip = get_ipython()
if ip:
    ip.register_magics(GlpkMagic)
else:
    logger.error("IPython environment not found. The GlpkMagic was not registered.")

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'glpk-5.0.tar.gz'


"Rozpoczynam kompilacje. Moze to potrwac kilka minut..."


The system cannot find the path specified.
'tee' is not recognized as an internal or external command,
operable program or batch file.


"Zakonczone"


## Testowanie instalacji

W przypadku pomyślnie zakończonej instalacji poniższe wykonanie powinno skutkować wyświetleniem informacji na temat wersji pakietu GLPK, np.:
`GLPSOL--GLPK LP/MIP Solver 5.0`

In [6]:
! glpsol -v

GLPSOL--GLPK LP/MIP Solver 5.0
Copyright (C) 2000-2020 Free Software Foundation, Inc.

This program has ABSOLUTELY NO WARRANTY.

This program is free software; you may re-distribute it under the terms
of the GNU General Public License version 3 or later.


## Instalacja -- Windows

Informacje oraz pliki `*.bat` dla systemu Windows znajdują się w poniższym katalogu.

Instalacja wymaga kompilatora `Microsoft Visual C++ v14`

In [7]:
!cd glpk-5.0/w64/ && cat readme.txt

The system cannot find the path specified.


## Dokumentacja

Oficjalna dokumentacja pakietu znajduje się w plikach `doc/glpk.pdf' oraz `doc/gmpl.pdf`.

Kopię można również pobrać, np. z fossies.org:
* [gmpl.pdf](https://fossies.org/linux/glpk/doc/gmpl.pdf) -- dokumentacja języka MathProg
* [glpk.pdf](https://fossies.org/linux/glpk/doc/glpk.pdf) -- dokumentacja solvera GLPK

# Przykład 1.

Przykładowy model dotyczący produkcji zabawek, prezentowany na wykładzie.

**Uwaga**: Pierwszy wiersz `%%writefile m1.mod` mówi, że wszystko poniżej zostanie zapisane w pliku m1.mod (w bieżącym katalogu). Umożliwi nam to
przygotowanie pliku z programem dla solvera.

In [8]:
%%writefile m1.mod

var x1 >= 0; /* żołnierzyki */
var x2 >= 0; /* pociągi */

/* Funkcja celu: */
maximize zysk: 4*x1 + 3*x2;

/* Ograniczenia: */
s.t. Materials : 2*x1 + 4*x2 <= 220;
s.t. Labor     : 3*x1 + 2*x2 <= 150;

end;

Overwriting m1.mod


In [9]:
# Uruchamiamy solver podając parametr "-m zad1.mod" wskazując tym samym plik
# z modelem:

! glpsol -m m1.mod -o m1.sol

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m m1.mod -o m1.sol
Reading model section from m1.mod...
12 lines were read
Generating zysk...
Generating Materials...
Generating Labor...
Model has been successfully generated
GLPK Simplex Optimizer 5.0
3 rows, 2 columns, 6 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  4.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =   2.150000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (102319 bytes)
Writing basic solution to 'm1.sol'...


In [10]:
# Wyświetlamy plik z wynikiem:
!type m1.sol

Problem:    m1
Rows:       3
Columns:    2
Non-zeros:  6
Status:     OPTIMAL
Objective:  zysk = 215 (MAXimum)

   No.   Row name   St   Activity     Lower bound   Upper bound    Marginal
------ ------------ -- ------------- ------------- ------------- -------------
     1 zysk         B            215                             
     2 Materials    NU           220                         220         0.125 
     3 Labor        NU           150                         150          1.25 

   No. Column name  St   Activity     Lower bound   Upper bound    Marginal
------ ------------ -- ------------- ------------- ------------- -------------
     1 x1           B             20             0               
     2 x2           B             45             0               

Karush-Kuhn-Tucker optimality conditions:

KKT.PE: max.abs.err = 2.84e-14 on row 2
        max.rel.err = 6.44e-17 on row 2
        High quality

KKT.PB: max.abs.err = 0.00e+00 on row 0
        max.rel.err = 0.00e+00 on 

Jak widać optymalne rozwiązanie to: `x1 = 20` oraz `x2 = 45` dające zysk `215`.

### Cell magic

Dla ułatwienia procesu uruchamiania możemy zastosować *skrót* (cell magic) `%%glpsol` zdefiniowany na początku tego notatnika.

In [11]:
%%glpsol

# Uwaga (!) na średniki na końcach wierszy -> ;

var x1 >= 0; # Żołnierzyki
var x2 >= 0; # Pociągi

# Funkcja celu:
maximize z: 4*x1 + 3*x2;

# Ograniczenia:
s.t. Materials : 2*x1 + 4*x2 <= 220;
s.t. Labor     : 3*x1 + 2*x2 <= 150;

solve; # Uruchom solver

# Wydruki -- printf - formatowany
printf "Wynik: %g\n", z;
# display - automatyczne formatowanie
display z, x1, x2;

end;  # <- end na końcu

In [12]:
%%glpsol

# @title Zapis uogólniony

set TOY;  # Zbiór zabawek
# Parametry:
param material { TOY };
param labor    { TOY };
param profit   { TOY };

param avail_labor >= 0;
param avail_material >= 0;

# Zmienne (zamiast x_i)
var Make { TOY } >= 0;

# Funkcja celu
maximize Total_profit: sum{i in TOY} profit[i] * Make[i];

# Ograniczenia
s.t. Mat_used : sum{i in TOY} material[i] * Make[i] <= avail_material;
s.t. Lab_used : sum{i in TOY} labor[i] * Make[i] <= avail_labor;


data;  # --- Dane instancji problemu ---

set TOY := soldier train;

param material :=
  soldier  2
  train    4 ;

param labor :=
  soldier  3
  train    2 ;

param profit :=
  soldier  4
  train    3 ;

param avail_material := 220;
param avail_labor := 150;

end;  # <- koniec, ważne !

# Elementy języka GNU MathProg

### Parametry

* służą do określania współczynników oraz innych stałych na potrzeby modelu
* odpowiadają stałym oraz tablicom z języków takich jak C czy Java
* wartości parametrów nie ulegają zmianom


In [13]:
%%glpsol

 param N := 42;
 param PI := 3.141592653589793;
 param z := 2 ^ 10;  # 1024
 param temp >= -80.5 <= 55.5;  # Deklaracja param. - bez podania wart.
 param wynik, integer > 0;

 display N, PI, z, temp, wynik; # Wydrukuj wart.

 data; # sekcja z danymi lub osobny plik

 param temp := -22;  # Konkretna wartość dla temp
 param wynik := 123;

 end; # Koniec pliku z modelem

In [14]:
%%glpsol

# Model:

# Tablica 3 wartości indeksowana 0..2:
param zysk{0..2};  # domyślnie rzeczywiste

# Macierz 2 x 3 wart. nieujemnych całk.
# Indeksowana 1..2, 1..3
param punkty{1..2, 1..3} default 7, integer, >= 0;

display zysk, punkty[2,3];
printf "punkty[2,2] = %g\n", punkty[2,2];

# --- Sekcja danych od słowa 'data;' ---
data;

# Uwaga -- najpierw indeks, potem wartość
param zysk :=
0  100.50  # zysk[0]
1 2120.0   # zysk[1]
2 3123.0;  # zysk[2]

param punkty :
   1  2  3 :=  # indeksy kolumn
1  5 13 30     # pierwszy wiersz
2 20  . 15 ;   # . oznacza wart. domyślną

end;

In [15]:
%%glpsol

param N := 4;

# Parametr zdefiniowany rekurencyjnie -- liczba kombinacji k-elem.
# ze zbioru N elementowego

param comb{n in 0..N, k in 0..n} :=
    if k == 0 or k == n then
        1
    else
        comb[n-1, k-1] + comb[n-1, k];


display comb;

printf "Jest dokładnie %g sposobów aby wybrać 2 elementy spośród 4\n", comb[4,2];

end;

### Zbiory

* Zbiór zawiera zero lub więcej elementów, z których każdy
  jest uporządkowaną listą (krotką) złożoną z jednego lub więcej komponentów
  * Wszystkie elementy zbiory są różne
  * Wszystkie muszą mieć tą samą liczbę komponentów, np. są parami liczb
    -- liczba komponentów jest wymiarem zbioru
* Rozmiar zbioru można otrzymać za pomocą funkcji `card`


In [16]:
%%glpsol

set X := { 1, 2, 3 };  # pełna definicja zbioru
set Y;  # tylko deklaracja
set Coords := X cross Y;  # iloczyn kartezjański
set Parzyste := 2..10 by 2;
set Primes := { n in 2..1000: !(exists{ d in 2..n: d*d <= n } n mod d == 0) };
set Nodes;
set Arcs within (Nodes cross Nodes);

display X, Y, Coords;
display Parzyste;
printf 'L.pierwszych w zakresie 2..1000 jest: %g\n', card(Primes);

data;  # Sekcja danych

set Y := a b;  # Stałe symboliczne
set Nodes := A B C D;
set Arcs := (A,B) (B,C) (C,D) (D,B);

end;

In [17]:
%%glpsol

# Operacje na zbiorach

set A := {1, 2, 3};
set B := {3, 4, 5};

display A union B;    # {1, 2, 3, 4, 5}`
display A diff B;     # {1, 2}
display A symdiff B;  # {1, 2, 4, 5}
display A inter B;    # {3}
display A cross B;    # {(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,3),(3,4),(3,5)}

end;

In [18]:
%%glpsol

# Zbiory umożliwiają m.in. wygodne indeksowanie parametrów i~zmiennych.

set Foods;
set Vitamins;
param NutritionalValue{ Foods, Vitamins };

data;  # Sekcja danych

set Foods := Apple Egg;
set Vitamins := A C D;

param NutritionalValue :
      A      C     D  :=
Egg   0.16   0.0   0.002
Apple 0.003  4.6   0.0 ;

end;

# Przykład 2. Problem diety

* Problem diety polega na znalezieniu diety o minimalnym koszcie, tak aby
  wybrane produkty żywnościowe pokrywały niezbędne zapotrzebowanie na
  wartości odżywcze.
* Problem ,,przeciwny'' do planowania produkcji -- zamiast maksymalizacji
  zysku, minimalizacja kosztów

* Dane: ceny produktów, zawartość witamin w produktach -- procent
  zapotrzebowania dziennego na opakowanie
* Zmienne: ile opakowań poszczególnych produktów należy kupić, tj.
  $x_{\rm BEEF}, x_{\rm CHK}, \ldots$
* Cel: minimalizacja kosztu zakupu produktów
* Ograniczenia: pokrycie minimalnego zapotrzebowania na wartości odżywcze
  (witaminy) w kontekście tygodnia, czyli $\ge 7 \cdot 100\%$ dawka każdej witaminy


In [19]:
%%writefile diet.dat

# Dane umieszczamy w osobnym pliku:

# Wart. odżywcze
set NUTR := A B1 B2 C ;

# Rodzaje produktów
set FOOD := BEEF CHK TUR ;

# Koszt
param:     cost  :=
    BEEF   3.19
    CHK    2.59
    TUR    2.49  ;

# Zapotrzebowanie min i maks na wart. odżywcze
param:  n_min   n_max :=
    A       700     10000
    C       700     10000
    B1      700     10000
    B2      700     10000 ;

# Ile jest witamin w każdym produkcie
# (tr) oznacza transpozycję tablicy amt
param amt (tr):
            A   C   B1   B2  :=
    BEEF   60  20   10   15
    CHK     8   0   20   20
    TUR    60  20   15   10 ;


Writing diet.dat


In [20]:
%%glpsol -d diet.dat

set NUTR;  # Wart. odżywcze
set FOOD;  # Produkty
param cost {FOOD} > 0;

# Zapotrzebowanie na składniki odżywcze:
param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

# Ilość składników w każdym produkcie:
param amt {NUTR,FOOD} >= 0;

# Ile kupić każdego produktu:
var Buy {FOOD} >= 0, integer;

# Minimalizacja kosztów:
minimize Total_Cost: sum {j in FOOD} cost[j] * Buy[j];

# Przy spełnieniu wymagań na wart. odżywcze:
subject to Diet {vit in NUTR}:
    n_min[vit] <= sum {j in FOOD} amt[vit,j] * Buy[j] <= n_max[vit];

solve;

display Buy;

end;

# Zadanie 1.

Rozwiąż za pomocą GLPK następujący problem LP:

    maximize z = 50x + 120y
    x + 2y ≤ 100
    x + 3y ≤ 120
    x + y ≤ 110
    x, y ≥ 0

In [21]:
%%glpsol

# Tutaj państwa rozwiązanie...

RuntimeError: GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m C:\Users\megaz\AppData\Local\Temp\tmpaxo1ngp7\model.lp -o C:\Users\megaz\AppData\Local\Temp\tmpaxo1ngp7\model.sol
Reading model section from C:\Users\megaz\AppData\Local\Temp\tmpaxo1ngp7\model.lp...
C:\Users\megaz\AppData\Local\Temp\tmpaxo1ngp7\model.lp:2: empty model section not allowed
Context:                                                          _|_
MathProg model processing error



# Zadanie 2.


Firma produkuje dwa rodzaje pudełek: `A` i `B`;
Pudełka wymagają wykonania dwóch głównych operacji:
*cięcia i składania*.
Cena sprzedaży za pudełko `A` wynosi `6`, a za pudełko `B` wynosi `4`.

Pudełko rodzaju `A` wymaga `2 minut cięcia` i `3 minut składania`, podczas gdy każde pudełko `B` wymaga `3 minut cięcia` i `1 minuty składania`.

Dostępny czas pracy wynosi `120 minut dla maszyny do cięcia` i `60 minut dla maszyny do składania` pudełek.
Kierownik musi określić optymalne ilości obu rodzajów pudełek do wyprodukowania w celu maksymalizacji zysków.

# Zadanie 3.

Firma z *poprzedniego* zadania rozpoczęła produkcję trzeciego rodzaju pudełka,
tj. `C`, którego czasy cięcia i składania wynoszą odpowiednio:

Opracuj **uniwersalny** model pozwalający na łatwe dodawanie kolejnych rodzajów pudełek oraz odczyt z *osobnego pliku z danymi*.

Jakie jest optymalne rozwiązanie dla następujących danych:

    set Box := A B C;
    
    param :
       Cutting   Assembly Price :=
    A        2          3     6
    B        3          1     4
    C      1.5        2.0     4 ;
    
    param MaxCuttingTime := 120 ;
    param MaxAssemblyTime := 60

  
**Wskazówka**: zdefiniuj parametry `Cutting`, `Assembly`, `Price` indeksowane elementami zbioru `Box`, np. `param Assembly { Box } > 0;`

# Zadanie 4.

Kayla pracuje nie więcej niż 20 godzin tygodniowo. Otrzymuje 10 € za korepetycje i 7 € za opiekę nad dziećmi.
Chce poświęcić co najmniej 3 godziny, ale nie więcej niż 8 godzin tygodniowo na korepetycje. Znajdź **najlepszy** oraz **najgorszy** możliwy podział czasu pracy Kayli.

# Zadanie 5.

Powierzchnia fabryki stali wynosi $F~{\rm m}^2$.
Do fabryki należy zakupić maszyny i ustawić je obok siebie. Maszyna typu $M$ zajmuje $Z_M~{\rm m}^2$ powierzchni podłogi
oraz produkuje $W_M~{\rm m}$ arkuszy stali na minutę.

W fabryce powinno powstawać co najmniej $L$ metrów arkuszy stali na minutę.

**Ile maszyn poszczególnych typów należy zakupić, tak aby zminimalizować koszt ich zakupu, ale zapewnić minimalną wydajność produkcji?**

Koszt zakupu maszyny typu $M$ wynosi ${\it Cost}_M~\$$.


Ile wynosi odpowiedź dla następujących danych:

    F = 350
    L = 600
    M = {A, B}  /* Maszyny */
    Z[A] = 60
    Z[B] = 110
    W[A] = 100
    W[B] = 200
    Cost[A] = 22000
    Cost[B] = 48000